In [2]:
import pandas as pd
import numpy as np
import os

In [2]:
import re
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import ngrams
import string
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import ndcg_score
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from nltk.util import bigrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import string

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
columns = ['note','auteur', 'avis','assureur', 'produit', 'type', 'date_publication', 'date_exp', 'avis_en', 'avis_cor', 'avis_cor_en']
df = pd.DataFrame(columns=columns)

In [4]:
folder_path = "Traduction_avis_clients"  # Specify the folder path

# Get the current working directory
script_directory = os.getcwd()

# Create the full path to the folder
folder_full_path = os.path.join(script_directory, folder_path)

# Check if the folder exists
if os.path.exists(folder_full_path) and os.path.isdir(folder_full_path):
    # List all files in the folder
    files = os.listdir(folder_full_path)

    # Loop through each file
    for file_name in files:
        # Create the full path to the file
        file_path = os.path.join(folder_full_path, file_name)

        # Check if it's a file (not a subdirectory)
        if os.path.isfile(file_path):
            # Read the content of the file
            try:
                small_df=pd.read_excel(file_path)
                try:
                    df = pd.concat([df, small_df], ignore_index=True)
                except:
                    print('unable to read data from '+file_name)
            except:
                print("unable to open file "+file_name)
else:
    print(f"The specified folder '{folder_path}' does not exist.")


unable to open file ~$avis_1_traduit.xlsx
unable to open file ~$avis_2_traduit.xlsx


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34435 entries, 0 to 34434
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   note              24104 non-null  object
 1   auteur            34434 non-null  object
 2   avis              34435 non-null  object
 3   assureur          34435 non-null  object
 4   produit           34435 non-null  object
 5   type              34435 non-null  object
 6   date_publication  34435 non-null  object
 7   date_exp          34435 non-null  object
 8   avis_en           34434 non-null  object
 9   avis_cor          435 non-null    object
 10  avis_cor_en       431 non-null    object
dtypes: object(11)
memory usage: 2.9+ MB


In [6]:
df.isna().sum()

note                10331
auteur                  1
avis                    0
assureur                0
produit                 0
type                    0
date_publication        0
date_exp                0
avis_en                 1
avis_cor            34000
avis_cor_en         34004
dtype: int64

In [7]:
df.dropna(subset=['avis_en','auteur'], inplace=True)
df = df.drop(['auteur', 'avis_cor', 'avis_cor_en'], axis=1)

In [8]:
def clean_avis(val):
    # Replacing \n
    val = val.replace('\n','')
    # Suppresson special caracters
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    return sentence

In [9]:
df["avis"] = df["avis"].apply(clean_avis)
df["avis_en"] = df["avis_en"].apply(clean_avis)

In [10]:
STOP_WORDS = stopwords.words('english')
STOP_WORDS.append("")
STOP_WORDS.extend(stopwords.words('french'))

def clean_sentence(val):
    """ Cette fonction permet de supprimer les caractères spéciaux, les stop words et 
    les mots de taille inférieure à 2 """  
    # Remplacement des valeurs \n
    val = val.replace('\n','')
    # Suppression des caratères spéciaux
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")

    for word in list(sentence):
        if (word in STOP_WORDS) | (len(word)<=2):
            sentence.remove(word)
    return sentence

def text2TokenList(text):
    # Utilisation de clean_sentence
    cleaned_text = clean_sentence(text)
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    token_list = [lemmatizer.lemmatize(token) for token in cleaned_text]
    return token_list

In [11]:
def text2BigramList(text):
    cleaned_text = clean_sentence(text)
    lemmatizer = WordNetLemmatizer()
    token_list = [lemmatizer.lemmatize(token) for token in cleaned_text]
    bigram_list=[]
    # Generate bigrams
    bigram_list += [f'{token1}_{token2}' for token1, token2 in bigrams(token_list)]
    return bigram_list

In [12]:
# English tokens/bigrams cleaning
df['tokens_en'] = df['avis_en'].apply(text2TokenList)
df['bigrams_en'] = df['avis_en'].apply(text2BigramList)

In [13]:
# French tokens cleaning
df['tokens_fr'] = df['avis'].apply(text2TokenList)
df['bigrams_fr'] = df['avis'].apply(text2BigramList)

In [14]:
df.isna().sum()

note                10331
avis                    0
assureur                0
produit                 0
type                    0
date_publication        0
date_exp                0
avis_en                 0
tokens_en               0
bigrams_en              0
tokens_fr               0
bigrams_fr              0
dtype: int64

In [15]:
df_train = df[df['type'] == 'train']
df_test=df[df['type'] == 'test']

In [16]:
df_train.isna().sum()

note                0
avis                0
assureur            0
produit             0
type                0
date_publication    0
date_exp            0
avis_en             0
tokens_en           0
bigrams_en          0
tokens_fr           0
bigrams_fr          0
dtype: int64

In [18]:
print(df_test.isna().sum())
print(df_test.shape)

note                10331
avis                    0
assureur                0
produit                 0
type                    0
date_publication        0
date_exp                0
avis_en                 0
tokens_en               0
bigrams_en              0
tokens_fr               0
bigrams_fr              0
dtype: int64
(10331, 12)


In [19]:
df_test = df_test.drop(['note'], axis=1)

In [ ]:
df_train = df_train.drop(['type'], axis=1)
df_test = df_test.drop(['type'], axis=1)

In [7]:
df_train.to_csv('train_set.csv', index=False)
df_test.to_csv('test_set.csv', index=False)

In [32]:
# Identifying the top 20 most common words in the dataframe
from collections import Counter
all_tokens = [token for sublist in df['tokens_en'] for token in sublist]
token_counts = Counter(all_tokens)
most_common_token = token_counts.most_common(20)
print("Most Common Token (English reviews):", most_common_token)

Most Common Token (English reviews): [('insurance', 17187), ('service', 10739), ('price', 10672), ('year', 9128), ('contract', 8298), ('satisfied', 7985), ('customer', 6184), ('month', 5877), ('good', 5131), ('time', 5051), ('without', 4525), ('vehicle', 4507), ('car', 4068), ('well', 4032), ('take', 3779), ('recommend', 3777), ('file', 3776), ('phone', 3693), ('since', 3621), ('day', 3585)]


In [33]:
# Identifying the top 20 most common bigrams in the dataframe
all_bigrams = [bigram for sublist in df['bigrams_en'] for bigram in sublist]
bigram_counts = Counter(all_bigrams)
most_common_bigram = bigram_counts.most_common(10)
print("Most Common Bigram (English reviews):", most_common_bigram)

Most Common Bigram (English reviews): [('satisfied_service', 2259), ('customer_service', 2183), ('direct_insurance', 1652), ('satisfied_price', 893), ('insurance_company', 680), ('service_price', 628), ('recommend_insurance', 612), ('take_care', 606), ('price_suit', 600), ('car_insurance', 581)]


In [34]:
# Identifying the top 20 most common words in the dataframe
from collections import Counter
all_tokens = [token for sublist in df['tokens_fr'] for token in sublist]
token_counts = Counter(all_tokens)
most_common_token = token_counts.most_common(20)
print("Most Common Token (French reviews):", most_common_token)

Most Common Token (French reviews): [('assurance', 14124), ('très', 13257), ('jai', 12501), ('plus', 11749), ('service', 10769), ('prix', 8580), ('bien', 7177), ('contrat', 6981), ('cest', 6889), ('tout', 6781), ('depuis', 6749), ('cette', 6312), ('fait', 6197), ('mois', 6164), ('client', 6138), ('chez', 5893), ('satisfait', 5806), ('faire', 5413), ('sans', 5393), ('sinistre', 4832)]


In [35]:
# Identifying the top 20 most common bigrams in the dataframe
all_bigrams = [bigram for sublist in df['bigrams_fr'] for bigram in sublist]
bigram_counts = Counter(all_bigrams)
most_common_bigram = bigram_counts.most_common(10)
print("Most Common Bigram (French reviews) :", most_common_bigram)

Most Common Bigram (French reviews) : [('cette_assurance', 2270), ('service_client', 2241), ('satisfait_service', 2009), ('direct_assurance', 1911), ('prise_charge', 1353), ('très_satisfait', 1026), ('très_bien', 1012), ('depuis_an', 936), ('depuis_plus', 822), ('satisfaite_service', 756)]
